Big Picture
* Get the dataset (pairs of input and label)
* Forward Pass: input -> function (model) -> output (prediction) 
* Compute the loss 
* Backward Pass (i.e backprop)
* Update the parameters (i.e. weights and biases)

In [ ]:
from pathlib import Path
import requests

# get the dataset
DATA_PATH = Path('data')
PATH = DATA_PATH / 'mnist'

PATH.mkdir(parents=True, exist_ok=True)
URL = 'https://github.com/pytorch/tutorials/raw/main/_static/'
FILENAME = 'mnist.pkl.gz'

if not (PATH / FILENAME).exists():
    content = requests.get(URL + FILENAME).content
    (PATH / FILENAME).open('wb').write(content)
